In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
recording_row

Subject                                                                  M04002
duration_m                                                                   10
video_recording_id                                     24-05-01-13-26-43-110846
ephys_id                                                    2024-05-01_13-26-37
calibration_id                                         24-05-01-13-45-07-825493
video_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
ephys_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
calibration_location_on_o2    /n/groups/datta/tim_sainburg/datasets/chronic2...
samplerate                                                                  150
username                                                                 tis697
n_ephys_streams                                                               1
max_video_duration_m                                                         10
Name: 0, dtype: object

### Job specific

In [9]:
import yaml
import textwrap
import time

In [10]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline

2D_predictions	    continuous_features   size_normalization  triangulation
arena_alignment     egocentric_alignment  spikesorting	      triangulation2
camera_calibration  gimbal		  tensorrt
camera_sync	    openephys_sync	  tensorrt-2


In [11]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [12]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [35]:
def check_egocentric_alignment_completion(directory):
    return (directory / "completed.txt").exists()

In [14]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [15]:
# where the video data is located
recording_directory = Path(recording_row.calibration_location_on_o2) / recording_row.calibration_id

In [16]:
# where to save output
egocentric_alignment_output_directory = (
    output_directory / "egocentric_alignment" / recording_row.video_recording_id
)
egocentric_alignment_output_directory_rigid = egocentric_alignment_output_directory / 'rigid'
egocentric_alignment_output_directory_nonrigid = egocentric_alignment_output_directory / 'nonrigid'
egocentric_alignment_output_directory_rigid.mkdir(parents=True, exist_ok=True)
egocentric_alignment_output_directory_nonrigid.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

In [17]:
# check if sync successfully completed
if config["egocentric_alignment"]["recompute_completed"] == False:
    if (
        check_egocentric_alignment_completion(egocentric_alignment_output_directory_rigid) &
        check_egocentric_alignment_completion(egocentric_alignment_output_directory_nonrigid) 
    ):
        print("egocentric_alignment completed, quitting")
        # return
    else:
        print("egocentric_alignment incomplete, starting")

egocentric_alignment completed, quitting


In [18]:
predictions_3d_file = list(
    (output_directory / "size_normalization" / recording_row.video_recording_id).glob(
        "size_norm.*.mmap"
    )
)[0]

In [19]:
assert predictions_3d_file.exists()

In [20]:
duration_requested = convert_minutes_to_hms(
        recording_row.duration_m * config["o2"]["egocentric_alignment"]["o2_runtime_multiplier"]
    )
duration_requested

'00:10:00'

In [21]:
params = {
    "predictions_3d_file": predictions_3d_file.as_posix(),
    "egocentric_alignment_output_directory_rigid": egocentric_alignment_output_directory_rigid.as_posix(),
    "egocentric_alignment_output_directory_nonrigid": egocentric_alignment_output_directory_nonrigid.as_posix(),
}

In [22]:
# create the job runner
runner = O2Runner(
    job_name_prefix=f"{recording_row.video_recording_id}_egocentric_alignment",
    remote_job_directory=remote_job_directory,
    conda_env="/n/groups/datta/tim_sainburg/conda_envs/peromoseq",
    o2_username=recording_row.username,
    o2_server="login.o2.rc.hms.harvard.edu",
    job_params=params,
    o2_n_cpus=config["o2"]["egocentric_alignment"]["o2_n_cpus"],
    o2_memory=config["o2"]["egocentric_alignment"]["o2_memory"],
    o2_time_limit=duration_requested,
    o2_queue=config["o2"]["egocentric_alignment"]["o2_queue"],
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Auth banner: b'Problems logging in?\nUse your lower case HMS ID, like abc123, not ABC123.\nIf locked out, see:\nhttps://it.hms.harvard.edu/i-want/reset-password-or-unlock-your-hms-account\n'
INFO:paramiko.transport:Authentication (publickey) successful!


In [23]:
runner.python_script = textwrap.dedent(
f"""
# load params
import yaml
params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
config_file = "{config_file.as_posix()}"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.keypoints.alignment.egocentric_alignment import EgocentricAligner 
# run rigid alignment
arena_aligner = EgocentricAligner(
    predictions_3d_file = params['predictions_3d_file'],
    egocentric_alignment_output_directory = params['egocentric_alignment_output_directory_rigid'],
    alignment_method="rigid",
    **config["egocentric_alignment"]
)
arena_aligner.run()

# run nonrigid alignment
arena_aligner = EgocentricAligner(
    predictions_3d_file = params['predictions_3d_file'],
    egocentric_alignment_output_directory = params['egocentric_alignment_output_directory_nonrigid'],
    alignment_method="nonrigid",
    **config["egocentric_alignment"]
)
arena_aligner.run()
"""
)

In [24]:
print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035/24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.keypoints.alignment.egocentric_alignment import EgocentricAligner 
# run rigid alignment
arena_aligner = EgocentricAligner(
    predictions_3d_file = params['predictions_3d_file'],
    egocentric_alignment_output_directory = params['egocentric_alignment_output_directory_rigid'],
    alignment_method="rigid",
    **config["egocentric_alignment"]
)
arena_aligner.run()

# run nonrigid alignment
arena_aligner = EgocentricAligner(
    predictions_3d_file = params['predictions_3d_file

In [25]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Successfully created remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Writing job files to remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Transferring /tmp/tmprtuhpw4l to login.o2.rc.hms.harvard.edu:/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035/24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-

In [26]:
# wait until the job is finished
# 10000/60/24 = roughly 1 week
for i in range(10000):
    # check job status every n seconds
    status = runner.check_job_status()
    if status:
        break
    time.sleep(60)

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43744251
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Unknown job state: 
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43744251
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is currently running.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43744251
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job has finished successfully.


In [33]:
!ls {egocentric_alignment_output_directory/'nonrigid'}

completed.txt					    poses_after_alignment.png
egocentric_alignment_rigid.float32.90009x25x3.mmap  poses_before_alignment.png


In [34]:
!ls {egocentric_alignment_output_directory/'rigid'}

completed.txt					    poses_after_alignment.png
egocentric_alignment_rigid.float32.90009x25x3.mmap  poses_before_alignment.png


In [36]:
# check if sync successfully completed
if (
    check_egocentric_alignment_completion(egocentric_alignment_output_directory_rigid) &
    check_egocentric_alignment_completion(egocentric_alignment_output_directory_nonrigid) 
):
    print("egocentric_alignment completed successfully")
    # return
else:
    print("egocentric_alignment did not complete successfully")

egocentric_alignment completed successfully


In [29]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035/

24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.log
24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.params.yaml
24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.py
24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.sh


In [30]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_140144_608035/24-05-01-13-26-43-110846_egocentric_alignment_24-08-09-2024-01-45-071737.log